In [3]:
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime  
from nltk.tokenize import word_tokenize  
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind
import numpy as np
import re


import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer

from tweepy import Stream
from tweepy.streaming import StreamListener

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#python'])